In [2]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, BaseMessage
from typing import List, TypedDict
from dotenv import load_dotenv

load_dotenv()
# Set your OpenAI API Key
# os.environ["OPENAI_API_KEY"]  # Replace this

True

# Step 3: Load and Split Web Content

In [3]:
url = "https://sunnah.com/bukhari/1"  # Example doc page
loader = WebBaseLoader(url)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = splitter.split_documents(docs)

In [4]:
splits

[Document(metadata={'source': 'https://sunnah.com/bukhari/1', 'title': '\n\tRevelation - Sunnah.com - Sayings and Teachings of Prophet Muhammad (صلى الله عليه و سلم)\n  ', 'description': 'Hadith of the Prophet Muhammad (saws) in English and Arabic', 'language': 'No language found.'}, page_content="Revelation - Sunnah.com - Sayings and Teachings of Prophet Muhammad (صلى الله عليه و سلم)\n  \n\n\n\n\n\n\n\n\nContribute to our mission\nDonate Now\n\n\n\n\nQur'an |\nSunnah |\nPrayer Times |\nAudio\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\nSelect Collections"),
 Document(metadata={'source': 'https://sunnah.com/bukhari/1', 'title': '\n\tRevelation - Sunnah.com - Sayings and Teachings of Prophet Muhammad (صلى الله عليه و سلم)\n  ', 'description': 'Hadith of the Prophet Muhammad (saws) in English and Arabic', 'language': 'No language found.'}, page_content="Qur'an |\nSunnah |\nPrayer Times |\nAudio\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\nSelect C

# Create Embeddings and ChromaDB

In [5]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")  # OpenAI embeddings
vectorstore = Chroma.from_documents(splits, embedding_model)
retriever = vectorstore.as_retriever()

In [6]:
class AgentState(TypedDict):
    messages: List[BaseMessage]

In [ ]:
# def create_agent(system_prompt: str):
#     return ChatOpenAI(model="gpt-4-1106-preview", temperature=0.2).bind(system=system_prompt)

# planner = create_agent("You are a planner. Break down the user's task into clear development steps.")
# researcher = create_agent("You are a researcher. Use context to provide supporting technical info.")
# coder = create_agent("You are a coder. Write Python code to solve the task.")
# reviewer = create_agent("You are a reviewer. Improve and verify the Python code.")

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

# Define prompt
def create_agent(system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])
    llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.3)
    chain = prompt | llm
    return chain


# Define Safe Agent Node

In [17]:
def safe_node(agent, label):
    def node(state: AgentState):
        try:
            input_text = state["messages"][-1].content
            response = agent.invoke({"input": input_text})  # ✅ this works now
            state["messages"].append(HumanMessage(content=response.content))
            return {"messages": state["messages"]}
        except Exception as e:
            state["messages"].append(HumanMessage(content=f"{label} Error: {e}"))
            return {"messages": state["messages"]}
    return node


# Build LangGraph Workflow

In [18]:
builder = StateGraph(AgentState)

builder.add_node("planner", safe_node(planner, "Planner"))
builder.add_node("researcher", safe_node(researcher, "Researcher", use_context=True))
builder.add_node("coder", safe_node(coder, "Coder"))
builder.add_node("reviewer", safe_node(reviewer, "Reviewer"))

builder.set_entry_point("planner")
builder.add_edge("planner", "researcher")
builder.add_edge("researcher", "coder")
builder.add_edge("coder", "reviewer")
builder.add_edge("reviewer", END)

graph = builder.compile()


TypeError: safe_node() got an unexpected keyword argument 'use_context'

In [15]:
user_query = "What is sunnah all about?"

init_state = {"messages": [HumanMessage(content=user_query)]}
result = graph.invoke(init_state)

for msg in result["messages"]:
    sender = "User" if isinstance(msg, HumanMessage) else "AI"
    print(f"{sender}:\n{msg.content}\n{'-'*50}")


User:
What is sunnah all about?
--------------------------------------------------
User:
Planner Error: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.
--------------------------------------------------
User:
Researcher Error: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.
--------------------------------------------------
User:
Coder Error: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.
--------------------------------------------------
User:
Reviewer Error: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.
--------------------------------------------------


# **Let's Start Again**

In [19]:
# 🧠 Imports
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, BaseMessage
from typing import TypedDict, List
import os

# 🌐 Load webpage and extract content
url = "https://sunnah.com/bukhari/1"
loader = WebBaseLoader(url)
docs = loader.load()

# 🧠 Create vector store with OpenAI Embeddings
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embedding, persist_directory="./chromadb_nlp")
retriever = vectorstore.as_retriever()

# 🤖 Create a RAG chain (retriever + LLM)
llm = ChatOpenAI(model="gpt-4.1", temperature=0.3)
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 🧬 LangGraph state
class AgentState(TypedDict):
    messages: List[BaseMessage]

# 🧱 Node: RAG LLM agent
def rag_node(state: AgentState) -> AgentState:
    last_message = state["messages"][-1].content
    result = rag_chain.invoke({"query": last_message})
    state["messages"].append(HumanMessage(content=result["result"]))
    return {"messages": state["messages"]}

# 🔗 Graph: single-node RAG agent
builder = StateGraph(AgentState)
builder.add_node("rag_agent", rag_node)
builder.set_entry_point("rag_agent")
builder.add_edge("rag_agent", END)
graph = builder.compile()


/tmp/ipykernel_64264/2552843518.py:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [20]:
# 🚀 Run the RAG agent
user_query = "What is sunnah all about?"
init_state = {"messages": [HumanMessage(content=user_query)]}
result = graph.invoke(init_state)

# 📃 Display the conversation
for msg in result["messages"]:
    print(f"{msg.type.upper()}: {msg.content}")


HUMAN: What is sunnah all about?
HUMAN: **Sunnah** in Islam refers to the teachings, actions, sayings, and approvals of the Prophet Muhammad (ﷺ). It is the second primary source of Islamic law and guidance after the Qur’an.

### What Does Sunnah Include?
- **Sayings (Hadith):** What the Prophet said on various matters.
- **Actions:** What the Prophet did in his daily life, worship, dealings, etc.
- **Approvals:** Things done by others in his presence which he did not object to, indicating his approval.

### Why is Sunnah Important?
- **Explanation of the Qur’an:** The Sunnah explains, details, and sometimes adds to the general commands in the Qur’an.
- **Practical Example:** The Prophet’s life is considered the best example for Muslims to follow in worship, character, and dealings.
- **Source of Law:** Alongside the Qur’an, the Sunnah is used to derive Islamic rulings (Shariah).

### Examples of Sunnah
- **Prayer:** The Qur’an commands prayer, but the Sunnah shows exactly how to perfor

In [21]:
# 🚀 Run the RAG agent
user_query = "How the Divine Revelation started being revealed to Allah's Messenger?"
init_state = {"messages": [HumanMessage(content=user_query)]}
result = graph.invoke(init_state)

# 📃 Display the conversation
for msg in result["messages"]:
    print(f"{msg.type.upper()}: {msg.content}")


HUMAN: How the Divine Revelation started being revealed to Allah's Messenger?
HUMAN: The beginning of Divine Revelation to Allah’s Messenger Muhammad (ﷺ) is described in detail in Sahih al-Bukhari, Book 1, Hadith 3, narrated by Aisha (may Allah be pleased with her):

1. **Good Dreams:**  
   The very first form of revelation was that the Prophet (ﷺ) would see good dreams, and whatever he saw in his dreams would come true like the bright daylight.

2. **Love of Seclusion:**  
   Then, the love of seclusion was placed in his heart. He used to go into seclusion in the Cave of Hira, where he would worship Allah alone for several days at a time.

3. **First Revelation in Cave Hira:**  
   While he was in the Cave of Hira, the angel Jibril (Gabriel) came to him and said, “Read!” The Prophet (ﷺ) replied, “I do not know how to read.” The angel pressed him tightly three times and each time asked him to read. After the third time, the angel said:

   > “Read in the name of your Lord, who has cre